# Basic optimization algorithms

The spirit of this simple tutorial consists in learning how to write simple solution algorithms. For each algorithm, test that it works, using simple test functions whose solution is known.

__Write a function `fixed_point(f::Function, x0::Float64)` which computes the fixed point of `f` starting from initial point `x0`.__

In [2]:
function fixed_point(f::Function, x0::Float64)
    x_new = f(x0)
    return x_new
end

function fixed_point(f::Function, x0::Float64, tol = 1E-5, maxitr = 100)
    x_new = x0
    for i in 1:maxitr
        while ((x_new - x0 < tol) || (-(x_new - x0) < tol))
            x_new = f(x_new)
        end
    end
    return x_new
end

fixed_point (generic function with 3 methods)

__Write a function `bisection(f::Function, a::Float64, b::Float64)` which computes a zero of function `f` within `(a,b)` using a bisection method.__

In [3]:
# Test function
f(x) = exp(x) - 1.1

f (generic function with 1 method)

In [23]:
# function bisection(f::Function, a::Float64, b::Float64)
#     y0a = f(a)
#     y0b = f(b)
#     c = (a+b)/2
#     yc = f(c)

#     if yc == 0
#         break
#     elseif y0a * yc > 0 # R. half side of [a,b]
#         a = c
#         y0a = yc
#     else # L. half side of [a,b]
#         b = c 
#     end

#     return c
    
# end

function bisection_rec(f::Function, a::Float64, 
    b::Float64, Ncalls = 0, y0a = f(a), y0b = f(b); τ_ϵ=1E-8, maxcalls = 100)
    c = (a+b)/2
    yc = f(c)

    if Ncalls > maxcalls 
        return (;solution=c, Ncalls = Ncalls)
    end

    if abs(yc) < τ_ϵ
        return (;solution=c, Ncalls=Ncalls)
    elseif y0b  * yc < 0 # Solution in R side of [a,b]
        return bisection_rec(f, c, b, Ncalls+1, yc, y0b)
    elseif y0a * yc < 0
        return bisection_rec(f, a, c, Ncalls+1, y0a, yc)
    else
        error("No solution")
    end

    end

bisection_rec (generic function with 4 methods)

In [24]:
sol = bisection_rec(f, 0.0, 1.0)
@time bisection_rec(f, 0.0, 1.0)

  0.000008 seconds (230 allocations: 4.312 KiB)


(solution = 0.09531018137931824, Ncalls = 24)

In [25]:
import Pkg; Pkg.add("BenchmarkTools")
using BenchmarkTools

   Resolving package versions...
  No Changes to `~/.julia/environments/v1.7/Project.toml`
  No Changes to `~/.julia/environments/v1.7/Manifest.toml`


In [28]:
@benchmark bisection_rec(f, 0.0, 1.0)

BenchmarkTools.Trial: 10000 samples with 9 evaluations.
 Range (min … max):  2.442 μs … 211.451 μs  ┊ GC (min … max): 0.00% … 98.63%
 Time  (median):     2.642 μs               ┊ GC (median):    0.00%
 Time  (mean ± σ):   3.028 μs ±   5.377 μs  ┊ GC (mean ± σ):  4.96% ±  2.78%

  ▃▇█▇▄▂▂▂▄▅▄▃ ▂▄▂   ▁▂▃▃▂▁                                   ▂
  █████████████████▇███████▆▅▅▄▅▄▆▆▆▅▆▅▁▁▅▄▅▄▄▃▄▄▅▅▅▁▅▃▄▄▃▄▄▄ █
  2.44 μs      Histogram: log(frequency) by time      6.31 μs <

 Memory estimate: 4.31 KiB, allocs estimate: 230.

In [38]:
function bisection_itr(f::Function, a::Float64, b::Float64; τ_ϵ=1E-8, maxit = 100)
    c = (a+b)/2
    it = 0
    fa = f(a)
    fb = f(b)
    fc = f(c)

    if !(fa*fc <= 0 || fb * fc < 0)
        error("No solution")
    end

    while (abs(fc) >= τ_ϵ && (it < maxit))
        it += 1
        c = (a+b)/2
        fa = f(a)
        fb = f(b)
        fc = f(c)

        if fc*fa < 0
            a, b = a, c
            c = (a+b)/2
            fc = f(c)
        else
            a, b = c, b
            c = (a+b)/2
            fa = fc
            fc = f(c)
        end

    end
    return c
end

bisection_itr (generic function with 1 method)

In [39]:
@benchmark bisection_itr(f, 0.0, 1.0)

BenchmarkTools.Trial: 10000 samples with 191 evaluations.
 Range (min … max):  523.010 ns …  1.864 μs  ┊ GC (min … max): 0.00% … 0.00%
 Time  (median):     535.660 ns              ┊ GC (median):    0.00%
 Time  (mean ± σ):   559.189 ns ± 99.620 ns  ┊ GC (mean ± σ):  0.00% ± 0.00%

  ▇█▃          ▂ ▂              ▁                              ▁
  ███▇▇▆▁▅▄▆▆▆▅█▅█▆▅▅▇▅▆▆▆▇▇▆▇▆██▇▇█▆▆▆▅▅▅▆▅▅▅▅▅▅▅▆▆▅▆▅▆▆▅▄▄▅▅ █
  523 ns        Histogram: log(frequency) by time      1.05 μs <

 Memory estimate: 0 bytes, allocs estimate: 0.

__Write a function `golden(f::Function, a::Float64, b::Float64)` which computes a zero of function `f` within `(a,b)` using a golden ratio method.__

__Write a function `zero_newton(f::Function, x0::Float64)` which computes the zero of function `f` starting from initial point `x0`.__

In [42]:
function zero_newton_v1(f::Function, x0::Float64; maxit = 1000, ϵ_tol = 1E-8, η_tol = 1E-8)
    success = false
    for n in 1:maxit
        (f0, df0) = f(x0)
        Δ = -f0/df0
        x1 = x0 + Δ
        η = abs(x1-x0)
        ϵ = abs(f0)
        if (η <= η_tol)||(ϵ < ϵ_tol)
            success = true
            return x0, true, n
        end      
        x0 = x1  
    end
    return x0, false
end

zero_newton (generic function with 1 method)

In [59]:
f(x) = (x*(x-1.0), 2x-1)

function fun(x)
    a, b = x
    return (
        [a * (a - 1.0), a * exp(b) - b],
        [(2*a-1) 0;
        exp(b) (a*exp(b)-1)]

    )
end

fun (generic function with 1 method)

In [60]:
fun([0.0, 0])

([-0.0, 0.0], [-1.0 0.0; 1.0 -1.0])

In [61]:
import Pkg; Pkg.add("LinearAlgebra")
using LinearAlgebra

   Resolving package versions...
  No Changes to `~/.julia/environments/v1.7/Project.toml`
  No Changes to `~/.julia/environments/v1.7/Manifest.toml`


In [62]:
function zero_newton(f::Function, x0::Vector{Float64}; maxit = 1000, ϵ_tol = 1E-8, η_tol = 1E-8)
    success = false
    for n in 1:maxit
        (f0, J0) = f(x0)
        Δ = -J0\f0
        x1 = x0 + Δ
        η = maximum(abs, x1-x0) # maximum(abs(x1)-abs(x0)) # or could use norm, but have to import from LinearAlgebra norm(x1-x0)
        ϵ = maximum(abs, f0) # abs(f0)
        if (η <= η_tol)||(ϵ < ϵ_tol)
            success = true
            return (;solution=x0, converged=true, n=n)
        end      
        x0 = x1  
    end
    return (;solution=x0, converged=false, n=NaN)
end

zero_newton (generic function with 2 methods)

In [63]:
sol = zero_newton(fun, [0.1, 0.2])

(solution = [-5.396595270071815e-16, -3.47729042410563e-15], converged = true, n = 5)

In [65]:
using ForwardDiff

g(a, b) = [a*(a-1.0), a*exp(b) - b]
g(x) = g(x...)

g (generic function with 2 methods)

In [68]:
function fobj(x)
    return (ForwardDiff.jacobian(g, x))
end


fobj (generic function with 1 method)

__Add an option `zero_newton(f::Function, x0::Float64, backtracking=true)` which computes the zero of function `f` starting from initial point `x0` using backtracking in each iteration.__

__Write a function `min_gd(f::Function, x0::Float64)` which computes the minimum of function `f` using gradient descent. Assume `f` returns a scalar and a gradient.__

__Write a function `min_nr(f::Function, x0::Float64)` which computes the minimum of function `f` using Newton-Raphson method. Assume `f` returns a scalar, a gradient, and a hessian.__

__Write a method `zero_newton(f::Function, x0::Vector{Float64})` which computes the zero of a vector valued function `f` starting from initial point `x0`.__

    

__Add an method `zero_newton(f::Function, x0::Vector{Float64}, backtracking=true)` which computes the zero of function `f` starting from initial point `x0` using backtracking in each iteration.__

__Add a method `zero_newton(f::Function, x0::Vector{Float64}, backtracking=true, lb=Vector{Float64})` which computes the zero of function `f` starting from initial point `x0` taking complementarity constraint into account `x>=lb` using the Fischer-Burmeister method.__